# Rigorous estimates of p-laplacian's first eigenvalue

In [1]:
using DifferentialEquations
using Plots; gr()
using IntervalArithmetic
using LinearAlgebra
using LaTeXStrings
using PrettyTables

## Functions

- `plaplace_solve` - Numerical solution of p-laplace
- `polynomial_interpolation` - IA interpolation of given points by 3rd degree polynomials
- `cubic_natural_spline` - IA interpolation of given points with natural cubic spline
- `cubic_end_slope_spline` - IA interpolation of given points with end slope cubic spline
- `get_u1` - Rebuilds interval expression of U₁ˢ by integrating U₂ˢ.
- `der_cubic_spline` - First derivative of interval cubic spline
- `lower_estimate` - Interval lower estimate of first eigenvalue λ₁
- `upper_estimate` - Interval upper estimate of first eigenvalue λ₁

In [2]:
"""
    plaplace_solve(λi, p, n; u₂0=1.0, dom=(0.0, 1.0))

Numericaly solves p-Laplace equation using shooting method.

Arguments:
λᵢₙᵢₜ ... initial interval for λ₁
p    ... p of p-Laplacian
n    ... number of solution points
u₂0  ... initial condition for u₂
dom  ... domain

Return:
t, tᴵ    ... division points where the solution is interpolated,
            double precision value and its interval representation
U₁, U₁ᴵ  ... numerical approximation of u₁ at t, tᴵ
U₂, U₂ᴵ  ... numerical approximation of u₂ at t, tᴵ
Λ₁       ... numerical approximation of first eigenvalue λ₁
"""
function plaplace_solve(λᵢₙᵢₜ, p, n; u₂0=1.0, dom=(0.0, 1.0))
   
    function sl(du,u,P,t)
        λ, p = P
        du[1] = abs(u[2])^(1/(p-1)) * sign(u[2])
        du[2] = -λ * abs(u[1])^(p-1)*sign(u[1]) 
    end
    
    tl, tr = dom
    
    u0 = [0.0; u₂0;] # initial condition
    a, b = λᵢₙᵢₜ
    Λ₁ = (a + b)/2
    Δt = (tr-tl)/(n-1)
    e = 1e-12 # stop condition

    while (b-a) >= e
        prob = ODEProblem(sl, u0, dom, (Λ₁, p))
        sol = solve(prob, saveat=Δt, abstol=1e-8,reltol=1e-8)
        if sol(tr)[1] == 0
            break
        else
            probA = ODEProblem(sl, u0, dom, (a, p))
            solA = solve(probA, saveat=Δt, abstol=1e-8,reltol=1e-8)
            probS = ODEProblem(sl, u0, dom, (Λ₁, p))
            solS = solve(probS, saveat=Δt, abstol=1e-8,reltol=1e-8)
            if solA(tr)[1] * solS(tr)[1] < 0
                b = Λ₁
            else
                a = Λ₁
            end
            Λ₁ = (a+b)/2
        end
    end

    prob = ODEProblem(sl, u0, dom, (Λ₁, p))
    sol = solve(prob, saveat=Δt, abstol=1e-8,reltol=1e-8)
    
    t = collect(LinRange(0,1,n-1))
    tᴵ = [@interval(i) for i in t]
    U₁ = [u[1] for u in sol(t).u]
    U₁ᴵ = [@interval(u[1]) for u in sol(t).u]
    U₂ = [u[2] for u in sol(t).u]
    U₂ᴵ = [@interval(u[2]) for u in sol(t).u]

    # první derivace použitá pro polynomy
    Ud = []
    for u2 in U₂
        if u2 >= 0
           append!(Ud, u2^(1/(p-1))) 
        else
           append!(Ud, -(-u2)^(1/(p-1)))  
        end
    end
    
    Udᴵ = []
    for u2I in U₂ᴵ
        if u2I >= 0
           append!(Udᴵ, u2I^(1/(p-1))) 
        else
           append!(Udᴵ, -(-u2I)^(1/(p-1)))  
        end
    end

    return t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud, Udᴵ, Λ₁
end;

### Koeficienty lineární interpolace

Symbolicky proložíme lineární fci mezi 2 sousedními body numerického řešení ve formě:
$$ u(x) = Ax + B$$
Neznámé jsou $A,B$, hodnoty $u(x)$ a $x$ známe.  
Řešíme soustavu:
\begin{align}
Ax_1 + B &= u_1 \\
Ax_2 + B &= u_2 \\
\end{align}
Výsledkem jsou koeficienty:
\begin{align}
A &= \frac{u_1 - u_2}{x_1 - x_2} \\
B &= \frac{x_1u_2 - x_2u_1}{x_1 - x_2} \\
\end{align}

In [3]:
"""
   linear_interpolation(t, tᴵ, U₁ᴵ)

IA interpolation of given points by 3rd degree polynomials . 
Returns coefficients `csc_V` as well as interval values `V` 
of the polynomials.

Arguments:
t, tᴵ        ... division points
Uᴵ           ... interval values to be interpolated
"""

function linear_interpolation(t, tᴵ, U₁ᴵ)
   
    csc_V = Vector[]
 
    for i in 1:length(U₁ᴵ)-1
        a = b = c = d = 0
        c = (U₁ᴵ[i]-U₁ᴵ[i+1]) / (t[1]-t[2]) 
        d = (t[1]*U₁ᴵ[i+1]-t[2]*U₁ᴵ[i]) / (t[1]-t[2])
        
        append!(csc_V, [Interval{Float64}[a,b,c,d]])
    end
    
    # napočítat boxy výsledné fce
    V = Interval{Float64}[] 
    for i in 1:length(U₁ᴵ)-1
        x_int = t[i]..t[i+1]
        f(x) = csc_V[i][4] + (x-t[i])*csc_V[i][3]
        append!(V, f.(x_int))
    end
        
    return csc_V, V
end;

### Koeficienty kubického polynomu

Symbolický výpočet koeficientů kubického polynomu mezi 2 body. 

$$u(x) = Ax^3 + Bx^2 + Cx + D$$

Neznámé jsou $A, B, C, D$, naopak známe hodnotu $u(x)$ a $u'(x)$ v krajních bodech.  
Chceme tedy řešit následující soustavu:

\begin{align}
Ax_1^3 + Bx_1^2 + Cx_1 + D &= u_1 \\
Ax_2^3 + Bx_2^2 + Cx_2 + D &= u_2 \\
3Ax_1^2 + 2Bx_1 + C &= u_1' \\
3Ax_2^2 + 2Bx_2 + C &= u_2' \\
\end{align}

kde $u_1, u_1'$ jsou hodnoty vlevo a $u_2, u_2'$ hodnoty vpravo, $x_1$ odpovídá t[i] a $x_2$ t[i+1]. Řešením soustavy jsou koeficienty:

$$ A = \frac{- 2 u_{1} + u_1' x_{1} - u_1' x_{2} + 2 u_{2} + u_2' x_{1} - u_2' x_{2}}{\left(x_{1} - x_{2}\right)^{3}} $$


$$ B = \frac{u_2' - u_1' + 3A(x_1^2-x_2^2)}{2(x_2-x_1)} $$

$$ C = u_1'-3Ax_1^2-2Bx_1 $$

$$ D = u_1 -Ax_1^3 - Bx_1^2 - Cx_1 $$

    

In [4]:
"""
   polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ)

IA interpolation of given points by 3rd degree polynomials . 
Returns coefficients `csc_V` as well as interval values `V` 
of the polynomials.

Arguments:
t, tᴵ        ... division points
Uᴵ           ... interval values to be interpolated
Udᴵ          ... interval first derivative values at division points
"""

function polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ)
   
    csc_V = Vector[]
 
    for i in 1:length(Udᴵ)-1
        a = b = c = d = 0
        
        a = (2*(U₁ᴵ[i+1]-U₁ᴵ[i]) + (Udᴵ[i]+Udᴵ[i+1])*(t[1]-t[2]))/(t[1]-t[2])^3
        b = (Udᴵ[i+1] - Udᴵ[i] + 3*a*(t[1]^2-t[2]^2))/(2*(t[2]-t[1]))
        c = Udᴵ[i] - t[1]*(2*b+t[1]*3*a)
        d = U₁ᴵ[i] - t[1]*(c+t[1]*(b+t[1]*a))
        
        append!(csc_V, [Interval{Float64}[a,b,c,d]])
    end
    
    # napočítat boxy výsledné fce
    V = Interval{Float64}[] 
    for i in 1:length(Udᴵ)-1
        x_int = t[i]..t[i+1]
        f(x) = csc_V[i][4] + (x-t[i])*(csc_V[i][3] + 
            (x-t[i])*(csc_V[i][2] + csc_V[i][1]*(x-t[i])))
        append!(V, f.(x_int))
    end
        
    return csc_V, V
end;

In [5]:
"""
   cubic_natural_spline(t, tᴵ, U, Uᴵ, Uₗd2, Uᵣd2; ns=10)

IA interpolation of given points by natural cubic spline. 
Returns spline coefficients `csc_V` as well as interval values `V` 
of the spline function.

Arguments:
t, tᴵ        ... division points
U, Uᴵ        ... values to be interpolated
Uₗd2, Uᵣd2    ... left and right boundary values of second derivative
ns=10        ... number of division points for each single piece of spline
"""
function cubic_natural_spline(t, tᴵ, U, Uᴵ, Uₗd2, Uᵣd2; ns=10)
    # A matrix
    n=length(Uᴵ)
    dv = [4..4 for i in 1:n-2]
    ev = [1..1 for i in 1:n-3]
    A = Array(SymTridiagonal(dv,ev))
    A⁻¹ = inv(A)

    # right-hand side
    h = 1.0/(n-1)
    rhs = []
    for i in 3:length(Uᴵ)
        append!(rhs, 6/h^2 * (Uᴵ[i] - 2 * Uᴵ[i-1] + Uᴵ[i-2]))
    end
    
    rhs[1] = rhs[1]-Uₗd2
    rhs[end] = rhs[end]-Uᵣd2    

    # second derivatives vector 
    Ud2 = []
    append!(Ud2, @interval(Uₗd2))
    append!(Ud2, A⁻¹*rhs)
    append!(Ud2, @interval(Uᵣd2))

    # spline coefficients
    csc_V = Vector[]
    for i in 1:length(Uᴵ)-1
        a = b = c = d = 0
        a = (Ud2[i+1]-Ud2[i])/(6*h)
        b = Ud2[i]/2 
        c = (Uᴵ[i+1] - Uᴵ[i])/h - h*(2*Ud2[i]+Ud2[i+1])/6
        d = Uᴵ[i]
        append!(csc_V, [Interval{Float64}[a,b,c,d]])
    end 
    
    V = Interval{Float64}[] 
    for i in 1:length(Uᴵ)-1
        x_dom = t[i]..t[i+1]
        x_int = mince(x_dom,ns)
        f(x) = csc_V[i][4] + (x-t[i])*(csc_V[i][3] + 
            (x-t[i])*(csc_V[i][2] + csc_V[i][1]*(x-t[i])))
        append!(V, f.(x_int))
    end
        
    return csc_V, V
end;

In [6]:
"""
    cubic_end_slope_spline(t, tᴵ, U, Uᴵ, Uₗd1, Uᵣd1; ns=10)

IA interpolation of given points with end slope cubic spline. 
Returns spline coefficients `csc_V`as well as interval values `V`.

Arguments:
t, tᴵ        ... division points
U, Uᴵ        ... values to be interpolated
Uₗd1, Uᵣd1    ... left and right boundary values of first derivative
ns=10        ... number of division points for each single piece of spline
"""

function cubic_end_slope_spline(t, tᴵ, U, Uᴵ, Uₗd1, Uᵣd1; ns=10)
    # A matrix
    n=length(Uᴵ)
    dv = [4..4 for i in 1:n-2]
    ev = [1..1 for i in 1:n-3]
    A = Array(SymTridiagonal(dv,ev))
    A[1,1] = 3.5..3.5
    A[end,end] = 3.5..3.5
    A⁻¹ = inv(A)

    # right-hand side
    h = 1.0/(n-1)
    rhs = []
    for i in 3:length(Uᴵ)
        append!(rhs, 6/h^2 * (Uᴵ[i] - 2 * Uᴵ[i-1] + Uᴵ[i-2]))
    end
    
    rhs[1] = rhs[1] - 3/h * ( (Uᴵ[2]-Uᴵ[1])/h - Uₗd1)
    rhs[end] = rhs[end] - 3/h * (Uᵣd1 - (Uᴵ[end]-Uᴵ[end-1])/h)    

    # second derivatives vector 
    sol = A⁻¹*rhs
    Ud2 = []
    
    σ₀ = 3/h * ( (Uᴵ[2]-Uᴵ[1])/h - Uₗd1) - sol[1]/2
    σ₁ = 3/h * (Uᵣd1 - (Uᴵ[end]-Uᴵ[end-1])/h)   - sol[end]/2
    append!(Ud2, @interval(σ₀))
    append!(Ud2, sol)
    append!(Ud2, @interval(σ₁))

    # spline coefficients
    csc_V = Vector[]
    for i in 1:length(Uᴵ)-1
        a=b=c=d=0
        a = (Ud2[i+1]-Ud2[i])/(6*h)
        b = Ud2[i]/2 
        c = (Uᴵ[i+1] - Uᴵ[i])/h - h*(2*Ud2[i]+Ud2[i+1])/6
        d = Uᴵ[i]
        append!(csc_V, Vector[Interval{Float64}[a,b,c,d]])
    end 
    
    V = Interval{Float64}[]
    for i in 1:length(Uᴵ)-1
        x_dom = t[i]..t[i+1]
        x_int = mince(x_dom,ns)
        f(x) = csc_V[i][4] + (x-t[i])*(csc_V[i][3] + 
            (x-t[i])*(csc_V[i][2] + csc_V[i][1]*(x-t[i])))
        append!(V, f.(x_int))
    end
        
    return csc_V, V
end;

In [7]:
"""
    get_v1(p, V₂, t)

Rebuilds interval expression of `V₁` by integrating `V₂`.

Arguments:
p             ... p of p-Laplacian
V₂            ... intervals values of v₂
t             ... division points
"""
function get_v1(p, V₂, t)
    
    f(x) = abs(x)^(1/(p-1))*sign(x)
    ni = mince(0..1,length(V₂))

    V₁_tmp = Interval[0..0]
    for i in 1:length(V₂)
        append!(V₁_tmp, V₁_tmp[end] + f(V₂[i]) * diam(ni[i]))
    end
    
    V₁ = Interval[]
    for i in 1:length(V₁_tmp)-1
        append!(V₁, V₁_tmp[i] ∪ V₁_tmp[i+1])
    end
    
    V₁ = V₁ .- inf(minimum(V₁))
    
    return V₁
end;

In [8]:
"""
    der_cubic_spline(csc, t, tᴵ, ns)

Computes the first derivative `V` of a given interval cubic spline. 

Arguments:
csc           ... cubic spline coefficients interval representation
t, tᴵ         ... division points
ns            ... number of division points for each single piece of spline
"""
function der_cubic_spline(csc, t, tᴵ, ns)
    
    V_tmp = Interval[]
    csc_Vder = [ [@interval(3) * c[1], @interval(2) * c[2], c[3]] for c in csc ] 
    for i in 1:length(t)-1
        x_dom = t[i]..t[i+1] 
        x_int = mince(x_dom,ns)
        f(x) = csc_Vder[i][3] + (x-t[i])*(csc_Vder[i][2] + (x-t[i])*csc_Vder[i][1])
        append!(V_tmp, f.(x_int))
    end
    
    V = Interval[]
    for i in 1:length(V_tmp)-1
        append!(V, V_tmp[i] ∪ V_tmp[i+1])
    end
    append!(V, V_tmp[end])
        
    return V
end;

In [9]:
"""
    lower_estimate(V₂_der, V₁, p)

Returns guaranteed lower estimate of first eigenvalue `λ₁ˡᵒʷ` and interval values of -u'₂/u₁⁽ᵖ⁻¹⁾ in `Fˡᵒʷ`.

Arguments:
V₂_der  ... interval values of v₂'
V₁      ... interval values of v₁
"""
function lower_estimate(V₂_der, V₁, p)
    f(x,y) = -x / y^(p-1)
    λ₁_tmp = f.(V₂_der, V₁)

    Fˡᵒʷ = Interval[]
    for i in 1:length(λ₁_tmp)-1
        append!(Fˡᵒʷ, λ₁_tmp[i] ∪ λ₁_tmp[i+1])
    end
    append!(Fˡᵒʷ,λ₁_tmp[end])
    
    λ₁ˡᵒʷ = inf(minimum(Fˡᵒʷ))
    
    return λ₁ˡᵒʷ, Fˡᵒʷ
end;

In [10]:
"""
    upper_estimate(V₁, V₁_der, p)

Returns guaranteed upper estimate of the first eigenvalue `λ₁ᵘᵖ`.

Arguments:
p             ... p of p-Laplacian
V₁           ... interval values of v₁
V₁_der       ... interval values of v₁'
"""
function upper_estimate(V₁, V₁_der, p)
    
    f(x) = abs(x)^(p)
    ni = mince(0..1,length(V₁))

    numerator = 0..0
    for i in 1:length(V₁_der)
        numerator = numerator + f(V₁_der[i]) * diam(ni[i])
    end
    
    denominator = 0..0
    for i in 1:length(V₁)
        denominator = denominator + f(V₁[i]) * diam(ni[i])
    end

    λ₁ᵘᵖ = sup(numerator/denominator)

    return λ₁ᵘᵖ 
end;

## Numerical experiments 


### Error comparison for spline and no-spline versions

Results are presented for various values of $p$ and $\lambda_1$ upper estimate count with help of cubic spline ($\lambda_1^{up}$) and without spline, using polynomial interpolation ($\lambda_1^{up} \text{ns}$)

- $p \in [1.5, 3.0]$ 
- $n = 202$
- $ns = 101$


In [19]:
@time begin
    n = 20301 # number of division points
    ns = 1 # number of subdivision points of each subinterval
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ˡᵒʷs = Float64[]
    λ₁ˡᵒʷs_err = Float64[]

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    si = mince(0..1,(n-1)*ns)

    ps = 1.5:0.1:3.0
    for p in ps
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        r = 10/(n*ns)
        dom = (-r, r+1)

        ### lower estimate
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
        #ud1 = -Λ₁ * U₁[end]^(p-1)
        csc_V₂, V₂ = linear_interpolation(t, tᴵ, U₂ᴵ);
        V₁ = get_v1(p, V₂, t);
        V₂_der = der_cubic_spline(csc_V₂, t, tᴵ, ns);
        λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(V₂_der, V₁, p)
        append!(λ₁ˡᵒʷs, λ₁ˡᵒʷ)
        append!(λ₁ˡᵒʷs_err, λ₁ˡᵒʷ-λ₁)

        ### upper estimate
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        #csc_V₁, V₁ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, 0., 0., ns=ns);
        csc_V₁, V₁ = linear_interpolation(t, tᴵ, U₁ᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err, λ₁ᵘᵖ-λ₁)
    end
    
end

197.261050 seconds (929.70 M allocations: 186.184 GiB, 17.11% gc time, 0.01% compilation time)


In [20]:
pretty_table(hcat(ps, floor.(λ₁ˡᵒʷs, digits=6), λ₁ᵉˣᵃᶜᵗ.(ps), ceil.(λ₁ᵘᵖs, digits=6)), header=["p","λ₁ˡᵒʷ","λ₁","λ₁ᵘᵖ"], formatters = ft_printf("%5.6f", 2:4))

┌─────┬───────────┬───────────┬───────────┐
│   p │     λ₁ˡᵒʷ │        λ₁ │      λ₁ᵘᵖ │
├─────┼───────────┼───────────┼───────────┤
│ 1.5 │  5.316224 │  5.318718 │  5.319898 │
│ 1.6 │  6.073489 │  6.076626 │  6.077904 │
│ 1.7 │  6.898153 │  6.902030 │  6.903435 │
│ 1.8 │  7.798793 │  7.803521 │  7.805079 │
│ 1.9 │  8.784027 │  8.789731 │  8.791468 │
│ 2.0 │  9.862783 │  9.869604 │  9.871550 │
│ 2.1 │ 11.044482 │ 11.052580 │ 11.054763 │
│ 2.2 │ 12.339169 │ 12.348721 │ 12.351175 │
│ 2.3 │ 13.757624 │ 13.768830 │ 13.771591 │
│ 2.4 │ 15.311464 │ 15.324548 │ 15.327655 │
│ 2.5 │ 17.013237 │ 17.028449 │ 17.031945 │
│ 2.6 │ 18.876521 │ 18.894140 │ 18.898074 │
│ 2.7 │ 20.916022 │ 20.936359 │ 20.940783 │
│ 2.8 │ 23.147676 │ 23.171079 │ 23.176052 │
│ 2.9 │ 25.588765 │ 25.615619 │ 25.621206 │
│ 3.0 │ 28.258028 │ 28.288762 │ 28.295035 │
└─────┴───────────┴───────────┴───────────┘


In [11]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ˡᵒʷs = Float64[]
    λ₁ˡᵒʷs_err = Float64[]

    λ₁ˡᵒʷs_ns = Float64[]
    λ₁ˡᵒʷs_err_ns = Float64[]    
    
    λ₁ˡᵒʷs_lin = Float64[]
    λ₁ˡᵒʷs_err_lin = Float64[]  
    
    #si = mince(0..1,(n-1)*ns)

    ps = 1.5:0.1:3.0
    for p in ps
        
        n = 201 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        r = 10/(n*ns)
        dom = (-r, r+1)
        
        
        ### lower estimate
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud, Udᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
        ud1 = -Λ₁ * U₁[end]^(p-1)
        csc_V₂, V₂ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);
        V₁ = get_v1(p, V₂, t);
        V₂_der = der_cubic_spline(csc_V₂, t, tᴵ, ns);
        λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(V₂_der, V₁, p)
        append!(λ₁ˡᵒʷs, λ₁ˡᵒʷ)
        append!(λ₁ˡᵒʷs_err, λ₁ˡᵒʷ-λ₁)
        
        
#        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
#        ud1 = -Λ₁ * U₁[end]^(p-1) # nemužu tohle pouzit na cele pole, misto jen posledni hodnoty? --> tim bych ziskal derivaci
#        csc_V₂, V₂ = cubic_end_slope_spline(t, tᴵ, U₂, U₂ᴵ, ud1, ud1, ns=ns);
#        V₁ = get_v1(p, V₂, t);
#        V₂_der = der_cubic_spline(csc_V₂, t, tᴵ, ns);
#        λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(V₂_der, V₁, p)
#        append!(λ₁ˡᵒʷs, λ₁ˡᵒʷ)
#        append!(λ₁ˡᵒʷs_err, λ₁ˡᵒʷ-λ₁)
        
        n = n*ns
        ns=1
#       t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
#        U₁[end] = 0
#        U₁ᴵ[end] = 0..0
#        csc_V₁, V₁ = polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ);
#        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
#        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
#        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
#        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud, Udᴵ, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n, dom=dom); #, 
        #ud1 = -Λ₁ * U₁[end]^(p-1)
        csc_V₂, V₂ = linear_interpolation(t, tᴵ, U₂ᴵ);
        V₁ = get_v1(p, V₂, t);
        V₂_der = der_cubic_spline(csc_V₂, t, tᴵ, ns);
        λ₁ˡᵒʷ, Fˡᵒʷ = lower_estimate(V₂_der, V₁, p)
        append!(λ₁ˡᵒʷs_lin, λ₁ˡᵒʷ)
        append!(λ₁ˡᵒʷs_err_lin, λ₁ˡᵒʷ-λ₁)
        
    end
    
end

#pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs, digits=6), ceil.(λ₁ᵘᵖs_err, digits=6), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6), floor.(λ₁ᵘᵖs_lin, digits=6), ceil.(λ₁ᵘᵖs_err_lin, digits=6)), header=["p","λ₁","λ₁ᵘᵖ", "err", "λ₁ᵘᵖ ns", "err ns", "λ₁ᵘᵖ lin", "err lin"], formatters = ft_printf("%5.6f", 2:4))
pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ˡᵒʷs, digits=6), ceil.(λ₁ˡᵒʷs_err, digits=6), floor.(λ₁ˡᵒʷs_lin, digits=6), ceil.(λ₁ˡᵒʷs_err_lin, digits=6)), header=["p","λ₁","λ₁ˡᵒʷ", "err", "λ₁ˡᵒʷ lin", "err lin"], formatters = ft_printf("%5.6f", 2:4))

161.531990 seconds (676.84 M allocations: 103.743 GiB, 9.93% gc time, 0.10% compilation time)
┌─────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│   p │        λ₁ │     λ₁ˡᵒʷ │       err │ λ₁ˡᵒʷ lin │   err lin │
├─────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 1.5 │  5.318718 │  5.316213 │ -0.002504 │   5.31622 │ -0.002493 │
│ 1.6 │  6.076626 │  6.073440 │ -0.003185 │   6.07349 │ -0.003136 │
│ 1.7 │  6.902030 │  6.898054 │ -0.003975 │   6.89815 │ -0.003876 │
│ 1.8 │  7.803521 │  7.798633 │ -0.004887 │   7.79879 │ -0.004727 │
│ 1.9 │  8.789731 │  8.783792 │ -0.005938 │   8.78403 │ -0.005703 │
│ 2.0 │  9.869604 │  9.862458 │ -0.007145 │   9.86278 │  -0.00682 │
│ 2.1 │ 11.052580 │ 11.044050 │ -0.008528 │   11.0445 │ -0.008096 │
│ 2.2 │ 12.348721 │ 12.338611 │ -0.010109 │   12.3392 │ -0.009551 │
│ 2.3 │ 13.768830 │ 13.756919 │ -0.011911 │   13.7576 │ -0.011205 │
│ 2.4 │ 15.324548 │ 15.310114 │ -0.014433 │   15.3115 │ -0.013083 │
│ 2.5 │ 17.028449 │ 17

In [12]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    λ₁ᵘᵖs_lin = Float64[]
    λ₁ᵘᵖs_err_lin = Float64[]  
    
    #si = mince(0..1,(n-1)*ns)

    ps = 1.5:0.1:3.0
    for p in ps
        
        n = 201 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, 0., 0., 
            ns=ns);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err, λ₁ᵘᵖ-λ₁)
        
        n = n*ns
        ns=1
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = linear_interpolation(t, tᴵ, U₁ᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_lin, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_lin, λ₁ᵘᵖ-λ₁)
        
    end
    
end

pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs, digits=6), ceil.(λ₁ᵘᵖs_err, digits=6), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6), floor.(λ₁ᵘᵖs_lin, digits=6), ceil.(λ₁ᵘᵖs_err_lin, digits=6)), header=["p","λ₁","λ₁ᵘᵖ", "err", "λ₁ᵘᵖ ns", "err ns", "λ₁ᵘᵖ lin", "err lin"], formatters = ft_printf("%5.6f", 2:4))

230.090510 seconds (1.09 G allocations: 191.979 GiB, 10.71% gc time, 0.23% compilation time)
┌─────┬───────────┬───────────┬──────────┬─────────┬──────────┬──────────┬──────────┐
│   p │        λ₁ │      λ₁ᵘᵖ │      err │ λ₁ᵘᵖ ns │   err ns │ λ₁ᵘᵖ lin │  err lin │
├─────┼───────────┼───────────┼──────────┼─────────┼──────────┼──────────┼──────────┤
│ 1.5 │  5.318718 │  5.320711 │ 0.001994 │ 5.32068 │ 0.001966 │   5.3199 │  0.00118 │
│ 1.6 │  6.076626 │  6.078730 │ 0.002105 │  6.0787 │ 0.002076 │   6.0779 │ 0.001278 │
│ 1.7 │  6.902030 │  6.904291 │ 0.002261 │ 6.90426 │ 0.002231 │  6.90343 │ 0.001404 │
│ 1.8 │  7.803521 │  7.805977 │ 0.002456 │ 7.80594 │ 0.002423 │  7.80508 │ 0.001558 │
│ 1.9 │  8.789731 │  8.792419 │ 0.002689 │ 8.79238 │ 0.002652 │  8.79147 │ 0.001738 │
│ 2.0 │  9.869604 │  9.872563 │ 0.002959 │ 9.87252 │ 0.002918 │  9.87155 │ 0.001946 │
│ 2.1 │ 11.052580 │ 11.055848 │ 0.003270 │ 11.0558 │ 0.003224 │  11.0548 │ 0.002184 │
│ 2.2 │ 12.348721 │ 12.352343 │ 0.003623 │ 12.3

### Running times

__Spline version first__

- $p \in [2.0, ]$ 
- $n = 502$
- $ns = 101$

In [11]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 502 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, 0., 0., 
            ns=ns);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err, λ₁ᵘᵖ-λ₁)

    end
    
end
pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs, digits=6), ceil.(λ₁ᵘᵖs_err, digits=6)), header=["p","λ₁","λ₁ᵘᵖ", "err"], formatters = ft_printf("%5.6f", 2:4))

  2.146679 seconds (6.60 M allocations: 379.516 MiB, 3.65% gc time, 0.19% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │     λ₁ᵘᵖ │      err │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.870779 │ 0.001175 │
└─────┴──────────┴──────────┴──────────┘


- $p \in [2.0, ]$ 
- $n = 502$
- $ns = 10100$

In [32]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 502 # number of division points
        ns = 10100 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, 0., 0., 
            ns=ns);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err, λ₁ᵘᵖ-λ₁)

    end
    
end
pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs, digits=6), ceil.(λ₁ᵘᵖs_err, digits=6)), header=["p","λ₁","λ₁ᵘᵖ", "err"], formatters = ft_printf("%5.6f", 2:4))

 32.967742 seconds (496.35 M allocations: 19.170 GiB, 10.33% gc time, 0.29% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │     λ₁ᵘᵖ │      err │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.869616 │ 0.000012 │
└─────┴──────────┴──────────┴──────────┘


- $p \in [2.0, ]$ 
- $n = 5020$
- $ns = 101$

In [25]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 5020 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, 0., 0., 
            ns=ns);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err, λ₁ᵘᵖ-λ₁)

    end
    
end
pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs, digits=6), ceil.(λ₁ᵘᵖs_err, digits=6)), header=["p","λ₁","λ₁ᵘᵖ", "err"], formatters = ft_printf("%5.6f", 2:4))

1805.439877 seconds (129.37 M allocations: 6.976 GiB, 0.07% gc time, 0.00% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │     λ₁ᵘᵖ │      err │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.869721 │ 0.000117 │
└─────┴──────────┴──────────┴──────────┘


- $p \in [2.0, ]$ 
- $n = 1004$
- $ns = 505$

In [24]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 1004 # number of division points
        ns = 505 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
        
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, _, _, Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = cubic_natural_spline(t, tᴵ, U₁, U₁ᴵ, 0., 0., 
            ns=ns);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err, λ₁ᵘᵖ-λ₁)

    end
    
end
pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs, digits=6), ceil.(λ₁ᵘᵖs_err, digits=6)), header=["p","λ₁","λ₁ᵘᵖ", "err"], formatters = ft_printf("%5.6f", 2:4))

 17.079844 seconds (53.46 M allocations: 2.319 GiB, 2.34% gc time, 0.03% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │     λ₁ᵘᵖ │      err │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.869721 │ 0.000118 │
└─────┴──────────┴──────────┴──────────┘


__No-spline version second__

- $p \in [2.0, ]$ 
- $n = 50702$

In [16]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 502 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
       
        n = n*ns
        ns=1
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
    end
    
end

pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6)), header=["p","λ₁","λ₁ᵘᵖ ns", "err ns"], formatters = ft_printf("%5.6f", 2:4))

  6.943616 seconds (47.87 M allocations: 13.247 GiB, 29.37% gc time, 1.28% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │  λ₁ᵘᵖ ns │   err ns │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.870772 │ 0.001169 │
└─────┴──────────┴──────────┴──────────┘


In [14]:
@time begin
    ### LINEARNI INTERPOLACE 
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 502 # number of division points
#        ns = 1001 # number of subdivision points of each subinterval
        ns = 2001 # number of subdivision points of each subinterval        
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
       
        n = n*ns
        ns=1
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = linear_interpolation(t, tᴵ, U₁ᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
    end
    
end

pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6)), header=["p","λ₁","λ₁ᵘᵖ ns", "err ns"], formatters = ft_printf("%5.6f", 2:4))

151.235413 seconds (890.53 M allocations: 257.677 GiB, 39.57% gc time, 0.00% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │  λ₁ᵘᵖ ns │   err ns │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.869643 │ 0.000040 │
└─────┴──────────┴──────────┴──────────┘


- $p \in [2.0, ]$ 
- $n = 507020$

In [23]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    
    
    
    #si = mince(0..1,(n-1)*ns)

    ps = [2.0,]
    for p in ps
        
        n = 5020 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
       
        n = n*ns
        ns=1
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
    end
    
end

pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6)), header=["p","λ₁","λ₁ᵘᵖ ns", "err ns"], formatters = ft_printf("%5.6f", 2:4))

 74.194695 seconds (463.45 M allocations: 130.716 GiB, 38.87% gc time, 0.00% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │  λ₁ᵘᵖ ns │   err ns │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.869721 │ 0.000117 │
└─────┴──────────┴──────────┴──────────┘


- $p \in [2.0, ]$ 
- $n = 5070200$

In [26]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    

    ps = [2.0,]
    for p in ps
        
        n = 50200 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
       
        n = n*ns
        ns=1
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = polynomial_interpolation(t, tᴵ, U₁ᴵ, Udᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
    end
    
end

pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6)), header=["p","λ₁","λ₁ᵘᵖ ns", "err ns"], formatters = ft_printf("%5.6f", 2:4))

784.023623 seconds (4.63 G allocations: 1.275 TiB, 39.40% gc time, 0.00% compilation time)
┌─────┬──────────┬──────────┬──────────┐
│   p │       λ₁ │  λ₁ᵘᵖ ns │   err ns │
├─────┼──────────┼──────────┼──────────┤
│ 2.0 │ 9.869604 │ 9.869616 │ 0.000012 │
└─────┴──────────┴──────────┴──────────┘


In [ ]:
@time begin
    
    λ₁ᵉˣᵃᶜᵗ(P) = (P-1)*(2*(π/P)/(sin(π/P)))^P

    λ₁ᵘᵖs = Float64[]
    λ₁ᵘᵖs_err = Float64[]

    λ₁ᵘᵖs_ns = Float64[]
    λ₁ᵘᵖs_err_ns = Float64[]    

    ps = [2.0,]
    for p in ps
        
        n = 50200 # number of division points
        ns = 101 # number of subdivision points of each subinterval
        λ₁ = λ₁ᵉˣᵃᶜᵗ(p)
        λᵢₙᵢₜ = (3.,1.5*λ₁)
        
        ### upper estimate
       
        n = n*ns
        ns=1
        t, tᴵ, U₁, U₁ᴵ, U₂, U₂ᴵ, Ud , Udᴵ , Λ₁ = plaplace_solve(λᵢₙᵢₜ, p, n);
        U₁[end] = 0
        U₁ᴵ[end] = 0..0
        csc_V₁, V₁ = linear_interpolation(t, tᴵ, U₁ᴵ, Udᴵ);
        V₁_der = der_cubic_spline(csc_V₁, t, tᴵ, ns); 
        λ₁ᵘᵖ = upper_estimate(V₁, V₁_der, p);
        append!(λ₁ᵘᵖs_ns, λ₁ᵘᵖ)
        append!(λ₁ᵘᵖs_err_ns, λ₁ᵘᵖ-λ₁)
        
    end
    
end

pretty_table(hcat(ps, λ₁ᵉˣᵃᶜᵗ.(ps), floor.(λ₁ᵘᵖs_ns, digits=6), ceil.(λ₁ᵘᵖs_err_ns, digits=6)), header=["p","λ₁","λ₁ᵘᵖ ns", "err ns"], formatters = ft_printf("%5.6f", 2:4))